# Conclusion: Deploying and Customizing Your AI Agent

## Training Your AI Agent

To train your AI agent effectively, you need to update the **input files**. The key files that require customization are:

### 1. Policy Document
This file contains the structured operational guidelines that define the expected behavior of the AI agent. To adapt the agent to your use case, update the policy document located at:

📂 `notebooks/files/input/ecommerce_customer_service.md`

### 2. Test Cases
The test case file serves as the foundation for evaluating the agent’s performance. This file must contain structured test cases with the following required columns:

| **Column Name**        | **Description**                                            |
|------------------------|-----------------------------------------------------------|
| **Context**            | High-level user intent for the request.                   |
| **Request**            | The specific user query the agent must process.          |
| **Expected Function**  | The function the agent should invoke.                     |
| **Expected Inputs**    | JSON object defining the function’s required inputs.     |
| **Tool Input Columns** | Additional columns based on tool parameters (e.g., `Order ID`, `Reason`, `Refund Amount`, `Customer ID`). |

Example:

```csv
Context,Request,Expected Function,Expected Inputs,Order ID,Modification Details,Reason,Refund Amount,Customer ID,Complaint Details,Issue Details
I want to check the status of my order.,What is the status of order 12345?,get_order_status,"{""order_id"": ""12345""}",12345,,,,,,
Can you tell me if my order has shipped?,Check the status of order 67890.,get_order_status,"{""order_id"": ""67890""}",67890,,,,,,
I need to change my shipping address.,Update shipping address for order 11122.,modify_order,"{""order_id"": ""11122"", ""modification_details"": ""New Address: 123 Main St""}",11122,New Address: 123 Main St,,,,
```

📂 `notebooks/files/input/ecommerce_test_cases.csv`

Updating this file allows you to train the agent for different customer scenarios.

## Setting Up the Tools

For the agent to function properly, you need tool definitions. Each function used by the agent must have an associated **tool file**, structured like this:

### Example: Order Status Tool
```python
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
E-Commerce Tool to get the order status.
"""

from typing import Dict
from pydantic import BaseModel, Field
from langchain.tools import StructuredTool

class OrderStatusInput(BaseModel):
    """Tool input schema"""
    order_id: str = Field(
        ..., description="The unique identifier of the order to retrieve its status."
    )

def get_order_status(order_id: str) -> str:
    """Retrieve the status of an order given its order ID."""
    status = _fetch_order_status(order_id)
    return f"The order {order_id} status: {status}."

def _fetch_order_status(order_id: str) -> str:
    """
    Mock function to fetch the status of an order.
    In a real implementation, this would query a database or API.
    """
    mock_order_data: Dict[str, str] = {
        "11928": "Shipped",
        "22345": "Processing",
        "33456": "Delivered",
        "44567": "Cancelled"
    }
    return mock_order_data.get(order_id, "Order ID not found.")

# Tool class that is automatically parsed to obtain the tool information for the meta-prompting
OrderStatusTool = StructuredTool(
    name="get_order_status",
    func=get_order_status,
    args_schema=OrderStatusInput,
    description=(
        "Fetch the current status of an order given its order ID. "
        "Possible statuses: Processing, Shipped, Delivered, Cancelled."
    )
)

if __name__ == "__main__":
    TEST_ORDER_ID = "11928"
    response = get_order_status(TEST_ORDER_ID)
    print(response)
```

📂 `notebooks/meta_prompting/tools/get_order_status.py`

Each tool should be properly structured, even if it is **mocked** (simulated), since we are training the **routing capabilities** of the agent, not its internal functions.

## Configuring the Agent in `config.yaml`

Ensure that the configuration file is properly set up to integrate your LLMs, tools and training data. 

📂 `notebooks/config.yaml`

### Defining the Tools in `config.yaml`
The list of tools that the agent can use must be explicitly defined in the `config.yaml` file. Ensure all necessary tools are included under the `tools.modules` section:

```yaml
tools:
  path: notebooks.meta_prompting.tools
  close_function: close_case
  modules:
    - get_order_status
    - modify_order
    - cancel_order
    - track_shipment
    - initiate_return
    - check_refund
    - process_refund
    - order_history
    - handle_complaint
    - escalate_to_human
    - update_account
    - get_faq
    - case_resolution
```

If a tool is missing from this list, the agent will not be able to invoke it.

### Choosing the Right LLM for Meta-Prompting
It is important to use an LLM with strong **reasoning capabilities** to effectively perform meta-prompting. Depending on the LLM you are using, you may need to customize the **system prompts** in `config.yaml`. For example, **Claude** works better when system prompts are structured in **XML format**, while other models like GPT-4o might perform best with JSON or natural language-based prompts.

Ensure that the `config.yaml` file is properly set up to integrate your tools and training data. Below is an example of the correct configuration:

```yaml
prompts:
  type: JinjaTemplate
  environment: notebooks/meta_prompting/prompts/
  templates:
    customer_persona: customer_persona.txt
    agent_persona: agent_persona.txt
    routine_generation: routine_generation.txt
    routine_improvement: routine_improvement.txt
    evals_history: evals_history.txt
```

For instnce, if you are using **Claude**, you might want to modify the routine's prompts to use XML.

## Creating a Real Agent with LAT-Mesh

To integrate your agent into a chatbot, you can use **LAT-Mesh** services such as the **ToolRepository** and **ReasoningEngine**.

### Step 1: Creating the Tool Repository
```python
from athon.agents import ToolRepository

# Example configuration for the Tool Repository
REPO_CONFIG = {
    'type': 'LangChainStructured'
}

# Initialize the Tool Repository with the provided configuration
tool_repository = ToolRepository.create(REPO_CONFIG)

tools_objects, tools_metadata = load_tools(tools_path, tools_modules)
for tool_object, tool_metadata in zip(tools_objects, tools_metadata):
    tool_repository.add_tool(tool_object, tool_metadata)
```

### Step 2: Creating the ReAct Agent (Reasoning Engine)
```python
from athon.agents import ReasoningEngine

# Example configuration for the Reasoning Engine
ENGINE_CONFIG = {
    'type': 'LangChainForOpenAI',
    'system_prompt': 'PASTE_LATEST_META_PROMPT_HERE',
    'model': {
        'type': 'LangChainChatOpenAI',
        'api_key': 'your-api-key-here',
        'model_name': 'gpt-4o'
    },
    'memory': {
        'type': 'LangChainBufferMemory',
        'memory_key': 'chat_history'
    },
    'tools': {
        'type': 'LangChainStructured'
    },
    'verbose': True
}

# Initialize the Reasoning Engine with the provided configuration
reasoning_engine = ReasoningEngine.create(ENGINE_CONFIG)
```
Replace **`PASTE_LATEST_META_PROMPT_HERE`** with the **final improved meta-prompt** to ensure the agent follows its structured reasoning and tool usage effectively.

## Final Thoughts
By following these steps, you can **train, deploy, and fine-tune your AI agent** for real-world e-commerce customer service scenarios. If you need to expand its capabilities, simply:
- **Update the policy document** to define new rules.
- **Add test cases** to train the agent on different queries.
- **Ensure your tools are properly defined** and listed in `config.yaml`.
- **Customize the system prompts** in `config.yaml` to optimize performance for different LLMs.
- **Leverage LAT-Mesh services** to deploy a fully operational agent in your chatbot.

This structured approach ensures your AI agent remains **efficient, scalable, and adaptable** for evolving business needs. 🚀